In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci
from Wick import *
from math import factorial

In [2]:
norbs = 3
Nocc = 2
vacuumOccupations = [1 for i in range(Nocc)] + [0 for i in range (norbs - Nocc)]
print(vacuumOccupations)

[1, 1, 0]


In [3]:
oP = operatorProduct([basicOperator(2,0,1), basicOperator(1,0,1), basicOperator(2,1,1), basicOperator(1,1,1)])
nOP = normalOrder(oP, vacuumOccupations)
print(oP)
print(nOP)
print(wickExpand(oP, vacuumOccupations))
print(vacuumExpectationValue(oP, vacuumOccupations))

1.0 * a_{2\alpha}a_{1\alpha}a^{2\alpha}a^{1\alpha}
1.0 * a_{1\alpha}a^{2\alpha}a_{2\alpha}a^{1\alpha}
0
-1.0 * a_{1\alpha}a^{1\alpha}
0
0
0
0
0
0
0
-1.0 * a_{1\alpha}a^{1\alpha}
 + 1.0 * a_{1\alpha}a^{2\alpha}a_{2\alpha}a^{1\alpha}
0
-1.0 * a_{1\alpha}a^{1\alpha}
0
0
0
0
0
0
0
0.0


In [4]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [5]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [6]:
eri

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [7]:
mf.energy_elec() + mf.energy_nuc()

array([-1.10263881,  1.27706243])

In [8]:
mf.mo_coeff

array([[ 0.56092571, -1.10314488],
       [ 0.56092571,  1.10314488]])

In [9]:
mf.nelec

(1, 1)

In [10]:
def spinFreeSingleExcitation(p, q):
    summandList = []
    for spin in range(2):
        summandList.append(operatorProduct([basicOperator(p,1,spin), basicOperator(q,0,spin)]))
    return operatorSum(summandList)

In [11]:
def spinFreeDoubleExcitation(p, q, r, s):
    summandList = []
    for spin1 in range(2):
        for spin2 in range(2):
            summandList.append(operatorProduct([basicOperator(p,1,spin2), basicOperator(q,1,spin1), basicOperator(s,0,spin1), basicOperator(r,0,spin2)]))
    return operatorSum(summandList)

In [12]:
Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1 for i in range(Nocc)] + [0 for i in range (Norbs - Nocc)]


In [13]:
hamiltonian1Body = operatorSum([])
for p in range(Norbs):
    for q in range(Norbs):
        hamiltonian1Body = hamiltonian1Body + h1[p, q] * spinFreeSingleExcitation(p, q)

In [14]:
print(hamiltonian1Body)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}


In [15]:
print(wickExpand(hamiltonian1Body, [1,0]))

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0
-2.379701237214037
 + 1.1898506186070186 * a_{0\alpha}a^{0\alpha}
 + 1.1898506186070186 * a_{0\beta}a^{0\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [16]:
hamiltonian1Body

In [17]:
wickExpand(hamiltonian1Body, [1,0])

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0


In [18]:
for s in hamiltonian1Body.summandList:
    print(s)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
-1.1898506186070186 * a^{0\beta}a_{0\beta}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-0.5337491016607854 * a^{1\beta}a_{1\beta}


In [19]:
for s in normalOrder(hamiltonian1Body, [1,0]).summandList:
    print(s)

1.1898506186070186 * a_{0\alpha}a^{0\alpha}
1.1898506186070186 * a_{0\beta}a^{0\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-0.5337491016607854 * a^{1\beta}a_{1\beta}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [20]:
for s in wickExpand(hamiltonian1Body, [1,0]).summandList:
    print(s)

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0
-2.379701237214037
1.1898506186070186 * a_{0\alpha}a^{0\alpha}
1.1898506186070186 * a_{0\beta}a^{0\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-0.5337491016607854 * a^{1\beta}a_{1\beta}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [21]:
w = wickExpand(hamiltonian1Body, [1,0])

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0


In [22]:
w

In [23]:
print(w)

-2.379701237214037
 + 1.1898506186070186 * a_{0\alpha}a^{0\alpha}
 + 1.1898506186070186 * a_{0\beta}a^{0\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [24]:
vacuumExpectationValue(hamiltonian1Body, [1,0])

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0


-2.379701237214037

In [25]:
vEV = 0.
for summand in w.summandList:
    if summand.operatorList == []:
        print(summand)
        vEV += summand.prefactor


-2.379701237214037


In [26]:
vEV

-2.379701237214037

In [27]:
mf.energy_nuc()

0.6230529146661897

In [28]:
mf.energy_elec()

(-1.7256917258579134, 0.6540095113561237)

In [29]:
hamiltonian2Body = operatorSum([])
for p in range(Norbs):
    for q in range(p, Norbs):
        for r in range(Norbs):
            for s in range(r, Norbs):
                x = int(q + Norbs * p - 0.5 * p * (p + 1))
                y = int(s + Norbs * r - 0.5 * r * (r + 1))
                hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)

In [30]:
#for s in wickExpand(hamiltonian2Body, [1,0]).summandList:
 #   print(s)

In [31]:
vacuumExpectationValue(hamiltonian2Body, [1,0])

0
0
0.3270047556780619 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\beta}a_{0\beta}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a_{0\beta}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\alpha}a_{0\alpha}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0.6540095113561238

In [32]:
vacuumExpectationValue(hamiltonian1Body, [1,0]) + vacuumExpectationValue(hamiltonian2Body, [1,0])

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\beta}a_{0\beta}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a_{0\beta}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\alpha}a_{0\alpha}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


-1.7256917258579132

In [33]:
vacuumExpectationValue(hamiltonian2Body, [1,0])

0
0
0.3270047556780619 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\beta}a_{0\beta}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a_{0\beta}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\alpha}a_{0\alpha}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0.6540095113561238

In [34]:
mf.energy_elec()

(-1.7256917258579134, 0.6540095113561237)

In [35]:
hamiltonian = hamiltonian1Body + hamiltonian2Body

In [36]:
vacuumExpectationValue(hamiltonian, [1,0])

0
0
0
0
-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\beta}a_{0\beta}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\alpha}a_{0\alpha}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a_{0\beta}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


-1.7256917258579132

In [37]:
singlesAmplitudes = np.ones((Nocc, Norbs - Nocc))

In [38]:
doublesAmplitudes = np.ones((Nocc, Nocc, Norbs-Nocc, Norbs-Nocc))

In [39]:
doublesAmplitudes

array([[[[1.]]]])

In [40]:
newdoublesAmplitudes = deepcopy(doublesAmplitudes)
newdoublesAmplitudes[0,0,0,0] = -0.136380375 *0.5 / 0.990656547


In [41]:
clusterSingles = operatorSum([])
for a in range(Nocc):
    for k in range(Norbs - Nocc):
        r = k + Nocc
        clusterSingles = clusterSingles + singlesAmplitudes[a,k] * spinFreeSingleExcitation(r, a)

In [42]:
clusterDoubles = operatorSum([])
for a in range(Nocc):
    for b in range(Nocc):
        for k in range(Norbs - Nocc):
            r = k + Nocc
            for l in range(Norbs - Nocc):
                s = l + Nocc
                clusterDoubles = clusterDoubles + doublesAmplitudes[a,b,k,l] * spinFreeDoubleExcitation(r, s, a, b)
clusterDoubles.checkNilpotency()

In [43]:
newclusterDoubles = operatorSum([])
for a in range(Nocc):
    for b in range(Nocc):
        for k in range(Norbs - Nocc):
            r = k + Nocc
            for l in range(Norbs - Nocc):
                s = l + Nocc
                newclusterDoubles = newclusterDoubles + newdoublesAmplitudes[a,b,k,l] * spinFreeDoubleExcitation(r, s, a, b)
newclusterDoubles.checkNilpotency()

In [44]:
print(newclusterDoubles)

0.0
 + -0.06883332846938728 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332846938728 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [45]:
print(clusterSingles.summandList[0] * clusterDoubles.summandList[0])


0.0 * a^{1\alpha}a_{0\alpha}


In [46]:
print(clusterSingles)
print(clusterDoubles)
print(hamiltonian)

1.0 * a^{1\alpha}a_{0\alpha}
 + 1.0 * a^{1\beta}a_{0\beta}
0.0
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0
0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.3226247132942422 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 6.938893903907228e-17 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.33906809184265285 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.09376099045135429 *

In [47]:
hamiltonian.checkNilpotency()
print(hamiltonian)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.3226247132942422 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 6.938893903907228e-17 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.33906809184265285 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.09376099045135429 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{1\beta}a_{1\beta}a_

In [48]:
print(clusterSingles * hamiltonian * clusterSingles)

0.0
 + 0.0 * a^{1\alpha}a_{0\alpha}
 + 0.0 * a^{1\beta}a_{0\beta}
 + 0.0 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.0 * a^{1\beta}a_{0\beta}a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{1\beta}a^{1\beta}a_{0\beta}
 + -1.1898506186070186 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{1\alpha}a^{1\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{1\alpha}a_{0\alpha}a^{0\beta}a_{1\beta}a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{0\beta}a^{1\alpha}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{1\beta}a_{0\beta}a^{0\alpha}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + -1.1898506186070186 * a^{1\beta}a_{0\beta}a^{0\beta}a_{0\beta}a^{1\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{1\beta}a_{0\beta}a^{0\beta}a_{1\beta}a^{1\alpha}a_{0\alpha}
 + 2.60021255284

In [49]:
def exponentialOperator(operator, maxOrder):
    exponential = operatorSum([operatorProduct([], 1.)])
    for k in range(maxOrder):
        exponential += (1 / factorial(k + 1)) * pow(operator, k + 1)
    return exponential

In [50]:
print(exponentialOperator(clusterDoubles, 3))

1.0
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


In [51]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [52]:
print(commutator(clusterSingles, clusterSingles))

0.0
 + 0.0 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.0 * a^{1\beta}a_{0\beta}a^{1\alpha}a_{0\alpha}


In [53]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [54]:
print(BCHSimilarityTransform(hamiltonian, clusterSingles, 1))

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.0 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.0 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.3226247132942422 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.5337491016607854 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{1\beta}
 + 1.1898506186070186 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{0\alpha}
 + -2.600212552842601e-17 * a^{1\alpha}a_{0\alpha}a^{0\alpha

In [55]:
bch = BCHSimilarityTransform(hamiltonian, clusterDoubles, 4)
bch.checkNilpotency()
print(bch)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.3226247132942422 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 6.938893903907228e-17 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.33906809184265285 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.09376099045135429 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 5.204170427930421e-17

In [56]:
newbch = BCHSimilarityTransform(hamiltonian, newclusterDoubles, 4)
newbch.checkNilpotency()
print(newbch)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.3226247132942422 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + -0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 6.938893903907228e-17 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.33906809184265285 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.09376099045135429 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 5.204170427930421e-

In [57]:
for s in bch.summandList:
    print(s)
    print(vacuumExpectationValue2(s, vacuum))


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [58]:
print(bch.summandList[31])

1.1898506186070186 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}


In [59]:
energyEquation = vacuumExpectationValue2(bch, vacuum)
print(energyEquation)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [60]:
newenergyEquation = vacuumExpectationValue2(newbch, vacuum)
print(newenergyEquation)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [61]:
print(clusterDoubles)

0.0
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [62]:
projected = clusterDoubles.conjugate() * bch
projected.checkNilpotency()
print(projected)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.3226247132942422 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 6.938893903907228e-17 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.33906809184265285 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.09376099045135429 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{1\beta}a_{1\beta}a_

In [63]:
newprojected = newclusterDoubles.summandList[1].conjugate() * newbch
newprojected.checkNilpotency()
print(newprojected)

0.0
 + -0.0 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}
 + 3.2449653383791726e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a_{0\alpha}
 + 0.036739727234857224 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a_{1\alpha}
 + 3.2449653383791726e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a_{0\beta}
 + 0.036739727234857224 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a_{1\beta}
 + -0.02220733286252447 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -4.7762716330187526e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + -0.023339185339293696 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + -0.02220733286252447 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + -4.7762716330187526e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + -0.02333918533

In [71]:
print(getPositionsForMultipleContraction(newprojected.summandList[42], 8))
print(sumNFoldContractions(newprojected.summandList[42], 8))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [65]:
projectedAmplitudeEquation = vacuumExpectationValue2(projected, vacuum)
print(projectedAmplitudeEquation)

-0.4351928726809437


In [66]:
newprojectedAmplitudeEquation = vacuumExpectationValue2(newprojected, vacuum)
print(newprojectedAmplitudeEquation)

KeyboardInterrupt: 

In [64]:
hartreeFockElectronicEnergy = vacuumExpectationValue(hamiltonian, vacuum)
hartreeFockEnergy = hartreeFockElectronicEnergy + mol.energy_nuc()
print(hartreeFockEnergy)

0
0
0
0
-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\beta}a_{0\beta}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5.204170427930421e-17 * a^{1\alpha}a_{0\alpha}
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.09376099045135429 * a^{1\beta}a_{1\beta}
0
0
0
0
0
0
0
0
0.3270047556780619 * a^{0\alpha}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a_{0\beta}
0
0
0
0
0.3270047556780619
0
0
4.163336342344337e-17 * a^{0\alpha}a_{1\alpha}
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1.1026388111917234


In [65]:
hartreeFockEnergy + energyEquation

NameError: name 'energyEquation' is not defined

In [69]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [70]:
vacuumExpectationValue((1. + (-1.) * clusterDoubles) * hamiltonian * (1. + clusterDoubles), vacuum)

-0.4351928726809437

In [72]:
HT2 = hamiltonian * clusterDoubles
HT2.checkNilpotency()
print(HT2)

0.0
 + -0.0 * a^{1\alpha}a_{0\alpha}
 + -0.0 * a^{1\alpha}a_{1\alpha}
 + -0.0 * a^{1\beta}a_{0\beta}
 + -0.0 * a^{1\beta}a_{1\beta}
 + -0.0 * a^{0\alpha}a_{0\alpha}
 + 0.0 * a^{0\alpha}a_{1\alpha}
 + -0.0 * a^{0\beta}a_{0\beta}
 + 0.0 * a^{0\beta}a_{1\beta}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{

In [73]:
HT2CorrelationEnergy = vacuumExpectationValue2(HT2, vacuum)

In [74]:
print(HT2CorrelationEnergy)

1.2904988531769697


In [75]:
ECCD = hartreeFockElectronicEnergy + HT2CorrelationEnergy

In [76]:
ECCD

-0.43519287268094353

In [77]:
vacuumExpectationValue2(hamiltonian, vacuum)

-1.7256917258579132

In [78]:
newenergyEquation + mol.energy_nuc()

-1.1914681426418214

In [79]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [67]:
def contract2operators(o1, o2):
    if o1.quasi_cre_ann:
        return 0
    elif o2.quasi_cre_ann and (o1.spin == o2.spin):
        return int(o1.orbital == o2.orbital)
    else:
        return 0

In [75]:
def recursiveFullContraction(operatorProduct_, vacuum):
    operatorProduct_.applyFermiVacuum(vacuum)
    operatorList_ = operatorProduct_.operatorList
    if len(operatorList_) == 0:
        result = operatorProduct_.prefactor
    elif len(operatorList_) == 2:
        result = operatorProduct_.prefactor * contract2operators(operatorList_[0], operatorList_[1])
    elif len(operatorList_) % 2 == 0:
        result = 0
        for i in range(1, len(operatorList_) - 1):
            if contract2operators(operatorList_[0], operatorList_[i]):
                result += pow(-1, i-1) * recursiveFullContraction(operatorProduct(operatorList_[1:i] + operatorList_[i+1:], operatorProduct_.prefactor), vacuum)
            else:
                result += 0
        if contract2operators(operatorList_[0], operatorList_[-1]):
            result += recursiveFullContraction(operatorProduct(operatorList_[1:-1], operatorProduct_.prefactor), vacuum)
        else:
            result += 0
    return result
        


In [76]:
print(newprojected.summandList[42])
print(recursiveFullContraction(newprojected.summandList[42], vacuum))

0.00010521894510289679 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.00010521894510289679


In [78]:
recursivehartreeFockElectronicEnergy = 0
for product in hamiltonian.summandList:
    recursivehartreeFockElectronicEnergy += recursiveFullContraction(product, vacuum)
recursivehartreeFockEnergy = recursivehartreeFockElectronicEnergy + mol.energy_nuc()
print(recursivehartreeFockEnergy)

-1.1026388111917234


In [79]:
newprojected = newclusterDoubles.summandList[1].conjugate() * newbch
newprojected.checkNilpotency()
print(newprojected)

0.0
 + -0.0 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}
 + 3.2449653383791726e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a_{0\alpha}
 + 0.036739727234857224 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a_{1\alpha}
 + 3.2449653383791726e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a_{0\beta}
 + 0.036739727234857224 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a_{1\beta}
 + -0.02220733286252447 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -4.7762716330187526e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + -0.023339185339293696 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + -0.02220733286252447 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + -4.7762716330187526e-18 * a_{1\alpha}a_{1\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + -0.02333918533

In [81]:
recursiveFullContractedNewprojected = 0
for product in newprojected.summandList:
    recursiveFullContractedNewprojected += recursiveFullContraction(product, vacuum)
print(recursiveFullContractedNewprojected)

-0.03090978141715154
